# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [48]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [49]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


In [50]:
#further description;
print df.describe()
print df.std()

            admit         gre         gpa    prestige
count  397.000000  397.000000  397.000000  397.000000
mean     0.317380  587.858942    3.392242    2.488665
std      0.466044  115.717787    0.380208    0.947083
min      0.000000  220.000000    2.260000    1.000000
25%      0.000000  520.000000    3.130000    2.000000
50%      0.000000  580.000000    3.400000    2.000000
75%      1.000000  660.000000    3.670000    3.000000
max      1.000000  800.000000    4.000000    4.000000
admit         0.466044
gre         115.717787
gpa           0.380208
prestige      0.947083
dtype: float64


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [51]:
# frequency table for prestige and whether or not someone was admitted
print pd.crosstab(df.admit, df.prestige)

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [52]:
dummy_var = pd.get_dummies(df['prestige'], prefix='prestige')

In [59]:
dummy_var.columns=["prestige_1", "prestige_2", "prestige_3", "prestige_4"]
dummy_var.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: One less than total to prevent multicollinearity

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [60]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_var.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           0           1           0
1      1  660.0  3.67           0           0           1           0
2      1  800.0  4.00           1           0           0           0
3      1  640.0  3.19           0           0           0           1
4      0  520.0  2.93           0           0           0           1


In [63]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
print pd.crosstab(handCalc['admit'], handCalc['prestige_1'])

prestige_1    0   1
admit              
0           243  28
1            93  33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [68]:
print "There is a " + str(33./(33+28)) + " chance of getting admitted if you have attended a #1 ranked college"

There is a 0.540983606557 chance of getting admitted if you have attended a #1 ranked college


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [71]:
print "There is a " + str(93./(243+93)) + " chance of getting admitted if you have not attended a #1 ranked college"

There is a 0.276785714286 chance of getting admitted if you have not attended a #1 ranked college


#### 3.3 Calculate the odds ratio

In [75]:
print (33./(33+28))/(93./(243+93))

1.95452141724


#### 3.4 Write this finding in a sentenance: 

Answer: You are almost twice as likely to be admitted if you're prestige is one

#### 3.5 Print the cross tab for prestige_4

In [76]:
print pd.crosstab(handCalc['admit'], handCalc['prestige_4'])

prestige_4    0   1
admit              
0           216  55
1           114  12


#### 3.6 Calculate the OR 

In [78]:
print(12./(12+55)/(114./(114+216)))

0.518460329929


#### 3.7 Write this finding in a sentence

Answer: Odds go down if prestige is 4

## Part 4. Analysis

In [79]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_var.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           1           0
1      1  660.0  3.67           0           1           0
2      1  800.0  4.00           0           0           0
3      1  640.0  3.19           0           0           1
4      0  520.0  2.93           0           0           1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [80]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [81]:
train_cols = data.columns[1:]

#### 4.2 Fit the model

In [82]:
logit = sm.Logit(data['admit'], data[train_cols])
results = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [84]:
print results.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 05 Jun 2017   Pseudo R-squ.:                 0.08166
Time:                        15:18:51   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.0

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [86]:
print np.exp(results.params)

gre           1.002221
gpa           2.180027
prestige_2    0.506548
prestige_3    0.262192
prestige_4    0.211525
intercept     0.020716
dtype: float64


In [87]:
params =results.params
conf = results.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                2.5%     97.5%        OR
gre         1.000074  1.004372  1.002221
gpa         1.136120  4.183113  2.180027
prestige_2  0.272168  0.942767  0.506548
prestige_3  0.133377  0.515419  0.262192
prestige_4  0.093329  0.479411  0.211525
intercept   0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: 50 percent of applicants are admitted with a prestige of 2

#### 4.6 Interpret the OR of GPA

Answer: A an increase in gpa increases the odds of admittance almost two-fold

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [90]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [91]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [92]:
combos.head()

,0,1,2,3
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


#### 5.1 Recreate the dummy variables

In [95]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_vars = pd.get_dummies(combos['prestige'])
dummy_vars.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']




In [98]:
# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'intercept']
data = combos[cols_to_keep].join(dummy_vars.ix[:, 'prestige_2':])
print data.head()

     gre       gpa  intercept  prestige_2  prestige_3  prestige_4
0  220.0  2.260000        1.0           0           0           0
1  220.0  2.260000        1.0           1           0           0
2  220.0  2.260000        1.0           0           1           0
3  220.0  2.260000        1.0           0           0           1
4  220.0  2.453333        1.0           0           0           0


#### 5.2 Make predictions on the enumerated dataset

In [105]:
data['admit_pred'] = logit.fit().predict(data[train_cols])
print data.head()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6
     gre       gpa  intercept  prestige_2  prestige_3  prestige_4  admit_pred
0  220.0  2.260000        1.0           0           0           0    0.164173
1  220.0  2.260000        1.0           1           0           0    0.090492
2  220.0  2.260000        1.0           0           1           0    0.048977
3  220.0  2.260000        1.0           0           0           1    0.039890
4  220.0  2.453333        1.0           0           0           0    0.185907


#### 5.3 Interpret findings for the last 4 observations

Answer: For our 0th student they have a gre score of 220, gpa of 2.26 a prestige of 1 and a 16% chance of being accepted

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.